In [1]:
import pandas as pd
import numpy as np

# pandas와 모델 코드의 인터페이스

In [3]:
data = pd.DataFrame({'x0':[1,2,3,4,5],'x1':[0.01,-0.01,0.25,-4.1,0.],'y':[-1.5,0.,3.6,1.3,-2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [6]:
data.values   # DataFrame을 NumPy배열로 변환

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [7]:
df2 = pd.DataFrame(data.values, columns=['one','two','three'])
df2       # 다시 DataFrame으로 되돌리기

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [8]:
df3 = data.copy()
df3['strings'] = ['a','b','c','d','e']
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [9]:
# 전체 컬럼 중 일부만 사용하고자 할 때 --> loc를 이용해 values 속성에 접근

model_cols = ['x0','x1']
data.loc[:,model_cols].values    # y컬럼 제외하고 데이터들 출력

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [11]:
# Categorical형과 pandas.get_dummies 함수를 활용해 데이터셋에 숫자 아닌 컬럼이 있는 경우 처리해봅시다.

data['category'] = pd.Categorical(['a','b','a','a','b'], categories=['a','b'])

# category 컬럼을 더미값으로 치환하고 싶다면 더미값을 생성하고 category 컬럼을 삭제한 다음 결과와 합치면 됨

dummies = pd.get_dummies(data.category, prefix='category')
dummies

,category_a,category_b
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1


In [14]:
data_with_dummies = data.drop('category',axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


# Pastsy를 이용해서 모델링 생성하기

In [15]:
import patsy
# 통계 모델을 위한 라이브러리. 
# R이나 S 통계 프로그래밍 언어에서 사용하는 수식 문법과 비슷한 형식의 문자열 기반 '수식 문법'을 제공

In [16]:
data = pd.DataFrame({'x0':[1,2,3,4,5],'x1':[0.01,-0.01,0.25,-4.1,0.],'y':[-1.5,0.,3.6,1.3,-2.]})
y,X = patsy.dmatrices('y ~ x0 + x1',data)        # dmatrices 함수 주목
# a + b 문법은 a와 b를 더하라는 의미가 아니라 모델을 위해 생성된 배열을 설계하는 용법

y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [17]:
X                   # y와 X는 DesignMatrix 인스턴스

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [19]:
# DesignMatrix 인스턴스는 몇 가지 추가 데이터가 포함된 NumPy nadarray로 볼 수 있다.
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [22]:
np.asarray(X)        # astype처럼 객체의 형식을 ndarray로 변형시킴

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [25]:
# Intercept는 최소제곱(자승)회귀와 같은 선형 모델을 위한 표현, 모델에 0을 더해서 절편(Intercept)을 제거할 수 있다.

patsy.dmatrices('y ~ x0 + x1 + 0',data)

(DesignMatrix with shape (5, 1)
      y
   -1.5
    0.0
    3.6
    1.3
   -2.0
   Terms:
     'y' (column 0),
 DesignMatrix with shape (5, 2)
   x0     x1
    1   0.01
    2  -0.01
    3   0.25
    4  -4.10
    5   0.00
   Terms:
     'x0' (column 0)
     'x1' (column 1))

In [32]:
# 최소제곱회귀분석을 위해 numpy.linalg.lstsq 같은 알고리즘에 바로 넘길 수 있다.

coef,resid,_,_ = np.linalg.lstsq(X,y)    # linalg는 NumPy의 서브패키지
coef

C:\Users\Choi.YJ\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  This is separate from the ipykernel package so we can avoid doing imports until


array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [41]:
resid

array([19.63791494])

In [49]:
# 모델 메타 데이터는 design_info 속성을 통해 얻을 수 있는데 
# 예를 들면 모델의 컬럼명을 피팅된 항에 맞추어 Series를 만들어 낼 수 있다.

coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

Pastsy 용법으로 데이터 변환하기

In [50]:
# 여러 파이썬 코드와 혼용해 쓰기

y, X =  patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)',data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [58]:
y, X =  patsy.dmatrices('y ~ standardize(x0) + center(x1)',data)
# standardize는 표준화(평균 0, 분산 1), center는 센터링(평균값을 뺌)

X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [66]:
# patsy.build_design_matrices 함수는 입력으로 사용되는 원본 데이터셋에서 저장한 정보(이 경우 표준화,센터링)
# 를 사용해서 출력 데이터를 만들어내는 변환에 적용할 수 있다.

new_data = pd.DataFrame({'x0':[6,7,8,9], 'x1':[3.1,-0.5,0,2.3],'y':[1,2,3,4]})
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [69]:
# Patsy 문법에 더하기 기호(+)는 덧셈이 아니므로 데이터셋에서 이름으로 컬럼을 추가하고 싶다면 I라는 ㅌ특수한 함수 사용

y, X = patsy.dmatrices('y ~ I(x0 + x1)',data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

범주형 데이터와 Patsy

In [70]:
# Patsy에서 비산술 데이터를 사용하면 기본적으로 더미 변수로 변환된다. 
# intercept가 존재한다면 공선성을 피하기 위해 레벨 중 하나는 남겨두게 된다.

data = pd.DataFrame({'key1':['a','a','b','b','a','b','a','b'],'key2':[0,1,0,1,0,1,0,0],
                    'v1':[1,2,3,4,5,6,7,8],'v2':[-1,0,2.5,-0.5,4.0,-1.2,0.2,-1.7]})
y,X = patsy.dmatrices('v2 ~ key1',data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [72]:
# 모델에서 intercept를 생략한다면 각 범주값의 컬럼은 모델 설계 배열에 포함된다.

y,X = patsy.dmatrices('v2 ~ key1 + 0',data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [75]:
y,X = patsy.dmatrices('v2 ~ C(key2)',data)     # 산술 컬럼은 C 함수를 이용해 범주형으로 해석할 수도 있음
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [76]:
data['key2'] = data['key2'].map({0:'zero', 1:'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [78]:
y,X =  patsy.dmatrices('v2 ~ key1 + key2',data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [79]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)  # :(콜론)은 and의 의미
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# statsmodels

선형모델 예측하기

In [80]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# statsmodels에는 아주 기본적인 선형회귀 모델부터 좀 더 복잡한 선형회귀 모델까지 존재한다.

In [ ]:
# 어떻게 동작하는지 알기 위해 랜덤 데이터에서 선형 모델을 생성합시다.
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size
    return mean + np.sqrt(variance)*np.random.randn(*size)
# 동일한 난수 생성을 위해 시드값 지정
np.random.seed(12345)

N = 100
X = np.c_[dnorm(0,0.4,size=N),dnorm(0,0.6,size=N),dnorm(0,0.2,size=N)]
eps = dnorm(0,0.1,size=N)
beta = [0.1, 0.3, 0.5]

y=np.dot(X, beta) + eps

# 알려진 인자인 beta를 이용해 진짜 모델을 작성했다. 
# dnorm은 특정 평균과 분산을 가지는 정규분포 데이터를 생성하기 위한 도움 함수다.